# Games

Represents a game transition system (see [Principles of Model Checking, Def. 2.1]). 

$$
    G = \langle S, A, T, (AP, L), (F, Acc), \varphi \rangle,
$$


In the `Game` class, each component is represented as a function.

- The set of states $S$ is represented by `Game.states()` function,
- The set of actions $A$ is represented by `Game.actions()` function,
- The transition function $T$ is represented by `Game.delta(state, inp)` function,
- The set of atomic propositions is represented by `Game.atoms()` function,
- The labeling function $L$ is represented by `Game.label()` function,
- The final states $F$ is represented by `Game.final(state)` function.
- The acceptance condition $Acc$ is represented by `Game.acc_cond()` function.
- The logic-based objective $\varphi$ is represented by `Game.objective()` function.

All of the above functions are marked abstract. The recommended way to use `Game` class is by subclassing it and implementing its component functions.


## Categorization of a Game:

A game is categorized by three types:

#### Deterministic vs. Non-Deterministic vs. Probabilistic

A game can be either deterministic or non-deterministic or probabilistic.
To define a **deterministic** transition system, provide a keyword argument `is_deterministic=True` to the
constructor. To define a **nondeterministic** transition system, provide a keyword argument `is_deterministic=False`
to the constructor. To define a **probabilistic** transition system, provide a keyword arguments
`is_deterministic=False, is_probabilistic=True` to the constructor.

The design of `Game` class closely follows its mathematical definition.
Hence, the signatures of `delta` function for deterministic, nondeterministic, probabilistic games are different.

- **deterministic:**  `delta(state, act) -> single state`
- **non-deterministic:**  `delta(state, act) -> a list of states`
- **probabilistic:**  `delta(state, act) -> a distribution over states`

#### Turn-based vs. Concurrent 

A game can be turn-based or concurrent. To define a **concurrent** game, provide a keyword argument `is_turn_based=False`. The game is `turn_based` by default.

#### Number of players: 1/1.5/2/2.5 

A game can be a 1/1.5/2/2.5-player game. A one-player game models a deterministic motion planning-type problem in
a static environment. A 1.5-player game is an MDP. A two-player game models a deterministic interaction between
two strategic players. And, a 2.5-player game models a stochastic interaction between two strategic players.

If a game is one or two player, then the :py:meth:`Game.delta` is `deterministic`.
If a game is 1.5 or 2.5 player, then the :py:meth:`Game.delta` is either `non-deterministic` (when
transition probabilities are unknown), and `probabilistic` (when transition probabilities are known).

Every state in a turn-based game is controlled by a player. To define which player controls which state, define
a game component :py:meth:`Game.turn` which takes in a state and returns a value between 0 and 3 to indicate
which player controls the state.

In [1]:
# This code block is necessary only when using `ggsolver:v0.1` docker image.
import sys
sys.path.append('/home/ggsolver/')
sys.path.append('/home/jovyan/ggsolver/')

from examples.notebooks.jupyter_patch import *

In [2]:
import logging
import itertools 
logger = logging.getLogger()
# logger.setLevel(logging.ERROR)
logger.setLevel(logging.DEBUG)

In [3]:
import scipy.stats as stats 
import ggsolver.models as models

# Defining Games


There are two ways to define a game. 

1. Direct Instantiation
2. Parameterized Game Definition

We illustrate the two methods next for a deterministic two-player turn-based game.  



### Direct Instantiation: Two-player turn-based game (deterministic)
Consider a game defined by following parameters. 



In [4]:
states = list(range(3))
actions = ["a", "b"]
trans_dict = {
    0: {"a": 1, "b": 2},
    1: {"a": 1, "b": 1},
    2: {"a": 2, "b": 0},
}
atoms = [f"p{i}" for i in states]
label = {i: [f"p{i}"] for i in states}
turn = {
    0: 2,
    1: 2,
    2: 1,
}

To define a game, we instantiate `models.Game` class.

In [5]:
game = models.Game(states=states, actions=actions, trans_dict=trans_dict, atoms=atoms, label=label, turn=turn)
game

Internally, the defaults are used for defining the class of game. 

In [6]:
print(f"{game.is_deterministic()=}")
print(f"{game.is_probabilistic()=}")
print(f"{game.is_turn_based()=}")

game.is_deterministic()=True
game.is_probabilistic()=False
game.is_turn_based()=True


A transition system-like model is also defined automatically. 

In [7]:
print(f"{game.states()=}")
print(f"{game.actions()=}")
print(f"{game.atoms()=}")
print(f"{game.delta(0, 'a')=}")
print(f"{game.label(0)=}")
print(f"{game.turn(0)=}")

game.states()=[0, 1, 2]
game.actions()=['a', 'b']
game.atoms()=['p0', 'p1', 'p2']
game.delta(0, 'a')=1
game.label(0)=['p0']
game.turn(0)=2


### Direct Instantiation: Two-player turn-based game (non-deterministic) 

When defining a non-deterministic game, the transition function must return a list of next-states and the  `is_deterministic` should be set to `False`, and `is_probabilistic` flag should be set to `False`. 

In [8]:
states = list(range(3))
actions = ["a", "b"]
trans_dict = {
    0: {"a": [0, 1], "b": [2]},
    1: {"a": [1], "b": [1]},
    2: {"a": [2], "b": [0]},
}
atoms = [f"p{i}" for i in states]
label = {i: [f"p{i}"] for i in states}
turn = {
    0: 2,
    1: 2,
    2: 1,
}

In [9]:
game = models.Game(
    states=states, 
    actions=actions, 
    trans_dict=trans_dict, 
    atoms=atoms, 
    label=label, 
    turn=turn, 
    is_deterministic=False,
    is_probabilistic=False,
)
game

In [10]:
print(f"{game.is_deterministic()=}")
print(f"{game.is_probabilistic()=}")
print(f"{game.is_turn_based()=}")

game.is_deterministic()=False
game.is_probabilistic()=False
game.is_turn_based()=True


In [11]:
print(f"{game.states()=}")
print(f"{game.actions()=}")
print(f"{game.atoms()=}")
print(f"{game.delta(0, 'a')=}")
print(f"{game.label(0)=}")
print(f"{game.turn(0)=}")

game.states()=[0, 1, 2]
game.actions()=['a', 'b']
game.atoms()=['p0', 'p1', 'p2']
game.delta(0, 'a')=[0, 1]
game.label(0)=['p0']
game.turn(0)=2


### Direct Instantiation: Two-player turn-based game (probabilistic) 

When defining a stochastic game (quantitative probabilities are known), the transition function must return a distribution over next states and the  `is_deterministic` should be set to `False`, and `is_probabilistic` flag should be set to `True`. 

In [12]:
states = list(range(3))
actions = ["a", "b"]
trans_dict = {
    0: {"a": stats.rv_discrete([0, 1], [0.1, 0.9]), "b": stats.rv_discrete([0, 2], [0, 1])},
    1: {"a": [1], "b": [1]},
    2: {"a": [2], "b": [0]},
}
atoms = [f"p{i}" for i in states]
label = {i: [f"p{i}"] for i in states}
turn = {
    0: 2,
    1: 2,
    2: 1,
}

In [13]:
game = models.Game(
    states=states, 
    actions=actions, 
    trans_dict=trans_dict, 
    atoms=atoms, 
    label=label, 
    turn=turn, 
    is_deterministic=False,
    is_probabilistic=True,
)
game

In [14]:
print(f"{game.is_deterministic()=}")
print(f"{game.is_probabilistic()=}")
print(f"{game.is_turn_based()=}")

game.is_deterministic()=False
game.is_probabilistic()=True
game.is_turn_based()=True


In [15]:
print(f"{game.states()=}")
print(f"{game.actions()=}")
print(f"{game.atoms()=}")
print(f"{game.delta(0, 'a')=}")
print(f"{game.label(0)=}")
print(f"{game.turn(0)=}")

game.states()=[0, 1, 2]
game.actions()=['a', 'b']
game.atoms()=['p0', 'p1', 'p2']
game.delta(0, 'a')=<scipy.stats._distn_infrastructure.rv_discrete object at 0x7fcd48bde820>
game.label(0)=['p0']
game.turn(0)=2


### Parameterized Game Definition: Two-player turn-based game (deterministic) 

Many applications require a game definition that changes according to input parameters. For example, a two-player game in a gridworld is parameterized by the size of gridworld and goal cells of P1. In such cases, we derive a class from `Game` and specialize its methods. 

In [16]:
class Gridworld(models.Game):
    def __init__(self, rows, cols, goal, **kwargs):
        super(Gridworld, self).__init__(**kwargs)
        self._rows = rows
        self._cols = cols 
        self._goal = goal
        
    def states(self):
        return list(itertools.product(range(self._rows), range(self._cols), range(self._rows), range(self._cols), range(2)))
    
    def actions(self):
        return ["N", "E", "S", "W"]
    
    def delta(self, state, act):
        p1r, p1c, p2r, p2c, turn = state
        next_p1r, next_p1c = self.apply_action((p1r, p1c), act)
        next_p2r, next_p2c = self.apply_action((p2r, p2c), act)

        return next_p1r, next_p1c, next_p2r, next_p2c, 1 if turn == 2 else 2
    
    def atoms(self):
        return ["goal"] 
    
    def label(self, state):
        if state[:2] in self._goal:
            return ["goal"]
        return []
    
    def turn(self, state):
        return state[-1]
    
    def apply_action(self, cell, act):
        row, col = cell

        if act == "N":
            return (row + 1, col) if 0 <= row + 1 < self.dim[0] else (row, col)
        elif act == "E":
            return (row, col + 1) if 0 <= col + 1 < self.dim[1] else (row, col)
        elif act == "S":
            return (row - 1, col) if 0 <= row - 1 < self.dim[0] else (row, col)
        else:  # inp == "W":
            return (row, col - 1) if 0 <= col - 1 < self.dim[1] else (row, col)

In [17]:
game = Gridworld(rows=2, cols=2, goal=[(0, 0)])
game

In [18]:
print(f"{game.states()=}")
print(f"{game.actions()=}")
print(f"{game.atoms()=}")
print(f"{game.delta((0, 0, 1, 1), 'N')=}")
print(f"{game.label((0, 0, 1, 1))=}")
print(f"{game.turn((0, 0, 1, 1))=}")

TypeError: range expected at most 3 arguments, got 5